In [3]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import random


Data_for_Put_options = pd.read_excel('train_gld_before_2016_vola_2.xlsx', sheet_name='GLD_PUT')



seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
exp_dates_for_Put = Data_for_Put_options['Exp Date = T']
dates_for_Put = Data_for_Put_options['T - 5']
volas_for_Put_percent_21 = Data_for_Put_options['21_Day_Percent_Volatility']
volas_for_Put_nominal_21 = Data_for_Put_options['21_Day_Volatility']
volas_for_Put_percent_3 = Data_for_Put_options['3_Day_Percent_Volatility']
volas_for_Put_nominal_3 = Data_for_Put_options['3_Day_Volatility']
volas_for_Put_percent_9 = Data_for_Put_options['9_Day_Percent_Volatility']
volas_for_Put_nominal_9 = Data_for_Put_options['9_Day_Volatility']
volas_for_Put_percent_14 = Data_for_Put_options['14_Day_Percent_Volatility']
volas_for_Put_nominal_14 = Data_for_Put_options['14_Day_Volatility']
volas_for_Put_percent_30 = Data_for_Put_options['30_Day_Percent_Volatility']
volas_for_Put_nominal_30 = Data_for_Put_options['30_Day_Volatility']
volas_for_Put_percent_60 = Data_for_Put_options['60_Day_Percent_Volatility']
volas_for_Put_nominal_60 = Data_for_Put_options['60_Day_Volatility']
volas_for_Put_percent_90 = Data_for_Put_options['90_Day_Percent_Volatility']
volas_for_Put_nominal_90 = Data_for_Put_options['90_Day_Volatility']





Features_for_Put_options = Data_for_Put_options[['ETF Price on T - 5', 'Strike_1', 'Strike_2', 'Strike_3', 'Strike_4', 'Strike_5','90_Day_Percent_Volatility','90_Day_Volatility']].copy()

Close_prices_Put = Data_for_Put_options[['Close_1', 'Close_2', 'Close_3', 'Close_4', 'Close_5']]

Features_for_Put_options['ETF Price on T - 5'] = pd.to_numeric(Features_for_Put_options['ETF Price on T - 5'], errors='coerce')

Features_for_Put_options = Features_for_Put_options.dropna(subset=['ETF Price on T - 5'])

Close_prices_Put = Close_prices_Put.loc[Features_for_Put_options.index]


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Put_features_tensor = torch.tensor(Features_for_Put_options.values, dtype=torch.float32).to(device)
Put_target_tensor = torch.tensor(Close_prices_Put.values, dtype=torch.float32).to(device)

print("Features tensor shape:", Put_features_tensor.shape)
print("Target tensor shape:", Put_target_tensor.shape)

lr = 0.0010001

class MultiOutputOptionsPricingMLP(nn.Module):
    def __init__(self, in_Put_param, out_Put_param=100):
        super(MultiOutputOptionsPricingMLP, self).__init__()

        layer_sizes = [in_Put_param, 500, 500, 500, 500, 500, 500, 500, out_Put_param]  

        self.layers = nn.ModuleList()

        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))  
            if i < len(layer_sizes) - 2: 
                self.layers.append(nn.BatchNorm1d(layer_sizes[i + 1])) 
                self.layers.append(nn.ReLU())  

    def forward(self, x):
        for layer in self.layers:
            x = layer(x) 
        return x

input_features = Features_for_Put_options.shape[1]  
output_strikes = Close_prices_Put.shape[1] 
Model_Put_options = MultiOutputOptionsPricingMLP(input_features, output_strikes).to(device)

sample_input = torch.randn(5, input_features).to(device) 
output = Model_Put_options(sample_input)
print("Model output shape:", output.shape) 

Put_options_Criteria = nn.MSELoss() 
Optimizer_for_Puts = optim.SGD(Model_Put_options.parameters(), lr=lr)



Num_Epochs = 2000
for Epoch in range(Num_Epochs):
    Optimizer_for_Puts.zero_grad()
    
    Target_out_Put = Model_Put_options(Put_features_tensor)  

    Loss_for_Put_options = Put_options_Criteria(Target_out_Put, Put_target_tensor) 

    Loss_for_Put_options.backward()  
    Optimizer_for_Puts.step()  

    if (Epoch + 1) % 100 == 0:
        print(f'Epoch [{Epoch + 1}/{Num_Epochs}], Loss (Put): {Loss_for_Put_options.item():.4f}')

with torch.no_grad():
    Model_Put_options.eval()
    train_predictions_Put = Model_Put_options(Put_features_tensor).cpu().numpy()  

torch.save(Model_Put_options.state_dict(), "./training_results/Puts.pth")
def Metrics(actual, predicted):
    MSE = np.mean((actual - predicted) ** 2, axis=0)  
    MAE = np.mean(np.abs(actual - predicted), axis=0)  
    MAPE = np.mean(np.abs((actual - predicted) / actual), axis=0) * 100  
    return MSE, MAE, MAPE

MSE_Put, MAE_Put, MAPE_Put= Metrics(Close_prices_Put.values, train_predictions_Put)  
for i in range(len(MSE_Put)):
    print(f'Put Options Price {i+1}- Strike {i+1}: MSE (Training): {MSE_Put[i]:.4f}, MAE (Training): {MAE_Put[i]:.4f}, MAPE (Training): {MAPE_Put[i]:.4f}%')



Res_Put_options = pd.DataFrame({
    'Exp Date = T': exp_dates_for_Put,
    'T - 5': dates_for_Put,
    '21_Day_Percent_Volatility': volas_for_Put_percent_21,
    '21_Day_Volatility': volas_for_Put_nominal_21,
    '3_Day_Percent_Volatility': volas_for_Put_percent_3,
    '3_Day_Volatility': volas_for_Put_nominal_3,
    '9_Day_Percent_Volatility': volas_for_Put_percent_9,
    '9_Day_Volatility': volas_for_Put_nominal_9,
    '14_Day_Percent_Volatility': volas_for_Put_percent_14,
    '14_Day_Volatility': volas_for_Put_nominal_14,
    
    'ETF Price on T - 5': Features_for_Put_options['ETF Price on T - 5'].values,
    
    'Strike_1': Features_for_Put_options['Strike_1'].values,
    'Close_1': Data_for_Put_options['Close_1'].values,  # Actual close prices
    'Predicted Price_1': train_predictions_Put[:, 0],  # Predicted prices
    
    'Strike_2': Features_for_Put_options['Strike_2'].values,
    'Close_2': Data_for_Put_options['Close_2'].values,
    'Predicted Price_2': train_predictions_Put[:, 1],
    
    'Strike_3': Features_for_Put_options['Strike_3'].values,
    'Close_3': Data_for_Put_options['Close_3'].values,
    'Predicted Price_3': train_predictions_Put[:, 2],
    
    'Strike_4': Features_for_Put_options['Strike_4'].values,
    'Close_4': Data_for_Put_options['Close_4'].values,
    'Predicted Price_4': train_predictions_Put[:, 3],
    
    'Strike_5': Features_for_Put_options['Strike_5'].values,
    'Close_5': Data_for_Put_options['Close_5'].values,
    'Predicted Price_5': train_predictions_Put[:, 4],
    
})
pd.set_option('display.float_format', '{:.10f}'.format)

for i in range(1, 6):
    Res_Put_options[f'Price Prediction Difference_{i}'] = Res_Put_options[f'Predicted Price_{i}'] - Res_Put_options[f'Close_{i}']
    
    Res_Put_options[f'Price Prediction Difference in percentage_{i}'] = (
        Res_Put_options[f'Price Prediction Difference_{i}'] / Res_Put_options[f'Close_{i}'] * 100
    )

Res_Put_options['Average Price Prediction Difference'] = Res_Put_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1)

Res_Put_options['Average Price Prediction Difference in percentage'] = Res_Put_options[
    [f'Price Prediction Difference in percentage_{i}' for i in range(1, 6)]
].mean(axis=1)

Res_Put_options['Overall Average Price Prediction Difference (APPDN)'] = Res_Put_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1).mean()

Res_Put_options['Overall Average Price Prediction Difference in percentage (APPDP)'] = Res_Put_options[
    [f'Price Prediction Difference in percentage_{i}' for i in range(1, 6)]
].mean(axis=1).mean()
for i in range(1, 6): 
    Res_Put_options[f'MSE_Put_{i}'] = MSE_Put[i - 1]  
    Res_Put_options[f'MAE_Put_{i}'] = MAE_Put[i - 1] 
    Res_Put_options[f'MAPE_Put_{i}'] = MAPE_Put[i - 1] 

    

MSE_Put = ((Res_Put_options[[f'Close_{i}' for i in range(1, 6)]].values - 
              Res_Put_options[[f'Predicted Price_{i}' for i in range(1, 6)]].values) ** 2).mean().mean()
MAE_Put = (abs(Res_Put_options[[f'Close_{i}' for i in range(1, 6)]].values - 
             Res_Put_options[[f'Predicted Price_{i}' for i in range(1, 6)]].values)).mean().mean()
MAPE_Put = (100 * abs((Res_Put_options[[f'Close_{i}' for i in range(1, 6)]].values - 
               Res_Put_options[[f'Predicted Price_{i}' for i in range(1, 6)]].values) / 
               Res_Put_options[[f'Close_{i}' for i in range(1, 6)]].values)).mean().mean()

Res_Put_options['MSE'] = [MSE_Put] * len(Res_Put_options)  
Res_Put_options['MAE'] = [MAE_Put] * len(Res_Put_options) 
Res_Put_options['MAPE'] = [MAPE_Put] * len(Res_Put_options)  



print(Res_Put_options.head())
print(f'MSE: {MSE_Put}, MAE: {MAE_Put}, MAPE: {MAPE_Put}')

MSE_Put, MAE_Put, MAPE_Put = Metrics(Close_prices_Put.values, train_predictions_Put)

Res_Put_options['Learning rate'] = lr
Res_Put_options['Epoch size'] = Num_Epochs
Res_Put_options['Random seed'] = seed


for i in range(len(MSE_Put)):
    print(f'Put Options Price_{i+1} - Strike_{i+1}: MSE (Training): {MSE_Put[i]:.4f}, MAE (Training): {MAE_Put[i]:.4f}, MAPE (Training): {MAPE_Put[i]:.4f}%')


    

print(Res_Put_options.head())





with pd.ExcelWriter('./training_results/GLD_training_RMSprop_MLP_with_multiple_strikes_10600_with_90d__volatility_input_Puts.xlsx') as writer:
    Res_Put_options.to_excel(writer, sheet_name='Put_Options', index=False)





Features tensor shape: torch.Size([363, 8])
Target tensor shape: torch.Size([363, 5])
Model output shape: torch.Size([5, 5])
Epoch [100/2000], Loss (Put): 0.0463
Epoch [200/2000], Loss (Put): 0.0245
Epoch [300/2000], Loss (Put): 0.0173
Epoch [400/2000], Loss (Put): 0.0134
Epoch [500/2000], Loss (Put): 0.0108
Epoch [600/2000], Loss (Put): 0.0089
Epoch [700/2000], Loss (Put): 0.0079
Epoch [800/2000], Loss (Put): 0.0069
Epoch [900/2000], Loss (Put): 0.0062
Epoch [1000/2000], Loss (Put): 0.0055
Epoch [1100/2000], Loss (Put): 0.0053
Epoch [1200/2000], Loss (Put): 0.0048
Epoch [1300/2000], Loss (Put): 0.0045
Epoch [1400/2000], Loss (Put): 0.0042
Epoch [1500/2000], Loss (Put): 0.0040
Epoch [1600/2000], Loss (Put): 0.0037
Epoch [1700/2000], Loss (Put): 0.0035
Epoch [1800/2000], Loss (Put): 0.0034
Epoch [1900/2000], Loss (Put): 0.0031
Epoch [2000/2000], Loss (Put): 0.0031
Put Options Price 1- Strike 1: MSE (Training): 0.0066, MAE (Training): 0.0563, MAPE (Training): 11.7818%
Put Options Price 2

testing

In [4]:
import pandas as pd
import numpy as np
import torch

testing_Data_for_Put_options = pd.read_excel('test_gld_after_2016_vola_2.xlsx', sheet_name='GLD_PUT')
test_volas_for_Put_percent_3 = testing_Data_for_Put_options['3_Day_Percent_Volatility']
test_volas_for_Put_nominal_3 = testing_Data_for_Put_options['3_Day_Volatility']
test_volas_for_Put_percent_9 = testing_Data_for_Put_options['9_Day_Percent_Volatility']
test_volas_for_Put_nominal_9 = testing_Data_for_Put_options['9_Day_Volatility']
test_volas_for_Put_percent_14 = testing_Data_for_Put_options['14_Day_Percent_Volatility']
test_volas_for_Put_nominal_14 = testing_Data_for_Put_options['14_Day_Volatility']
test_volas_for_Put_percent_21 = testing_Data_for_Put_options['21_Day_Percent_Volatility']
test_volas_for_Put_nominal_21 = testing_Data_for_Put_options['21_Day_Volatility']
test_volas_for_Put_percent_30 = testing_Data_for_Put_options['30_Day_Percent_Volatility']
test_volas_for_Put_nominal_30 = testing_Data_for_Put_options['30_Day_Volatility']
test_volas_for_Put_percent_60 = testing_Data_for_Put_options['60_Day_Percent_Volatility']
test_volas_for_Put_nominal_60 = testing_Data_for_Put_options['60_Day_Volatility']
test_volas_for_Put_percent_90 = testing_Data_for_Put_options['90_Day_Percent_Volatility']
test_volas_for_Put_nominal_90 = testing_Data_for_Put_options['90_Day_Volatility']
test_dates_for_Put = testing_Data_for_Put_options['T - 5']
exp_dates_for_Put_test = testing_Data_for_Put_options['Exp Date = T']

testing_Data_for_Put_options.columns = testing_Data_for_Put_options.columns.str.strip()

testing_Features_for_Put_options = testing_Data_for_Put_options[['ETF Price on T - 5', 'Strike_1', 'Strike_2', 'Strike_3', 'Strike_4', 'Strike_5','90_Day_Percent_Volatility','90_Day_Volatility']].copy()
testing_Data_Target_Put = testing_Data_for_Put_options[['Close_1', 'Close_2', 'Close_3', 'Close_4', 'Close_5']]

testing_Features_for_Put_options['ETF Price on T - 5'] = pd.to_numeric(testing_Features_for_Put_options['ETF Price on T - 5'], errors='coerce')

testing_Features_for_Put_options = testing_Features_for_Put_options.dropna(subset=['ETF Price on T - 5'])

test_Close_prices_Put = testing_Data_Target_Put.loc[testing_Features_for_Put_options.index]

test_dates_for_Put = testing_Data_for_Put_options['T - 5']
exp_dates_for_Put_test = testing_Data_for_Put_options['Exp Date = T']

testing_Data_for_Put_options = testing_Data_for_Put_options.drop(columns=['Exp Date = T'])



Put_features_tensor_test = torch.tensor(testing_Features_for_Put_options.values, dtype=torch.float32).to(device)
Put_target_tensor_test = torch.tensor(testing_Data_Target_Put.values, dtype=torch.float32).to(device)

model_Put_loaded = MultiOutputOptionsPricingMLP(input_features, output_strikes).to(device)
model_Put_loaded.load_state_dict(torch.load('./training_results/Puts.pth'))
model_Put_loaded.eval()  

with torch.no_grad():  
    test_predictions_Put = model_Put_loaded(Put_features_tensor_test)

if isinstance(test_predictions_Put, torch.Tensor):
    test_predictions_Put = test_predictions_Put.cpu().numpy()  

assert test_Close_prices_Put.shape == test_predictions_Put.shape, "Shape mismatch between actual and predicted prices."

def Metrics(actual, predicted):
    MSE_test_Put = np.mean((actual - predicted) ** 2, axis=0)  
    MAE_test_Put = np.mean(np.abs(actual - predicted), axis=0)  
    MAPE_test_Put = np.mean(np.abs((actual - predicted) / (actual + 1e-8)), axis=0) * 100  
    
    return MSE_test_Put, MAE_test_Put, MAPE_test_Put

MSE_test_Put, MAE_test_Put, MAPE_test_Put= Metrics(test_Close_prices_Put.values, test_predictions_Put)

for i in range(len(MSE_test_Put)):
    print(f'Put Options Price_{i+1} - Strike_{i+1}: MSE (testing): {MSE_test_Put[i]:.4f}, MAE (testing): {MAE_test_Put[i]:.4f}, MAPE (testing): {MAPE_test_Put[i]:.2f}%')

Test_Put_options = pd.DataFrame({
    'Exp Date = T': exp_dates_for_Put_test,
    'T - 5': test_dates_for_Put,
    '3_Day_Percent_Volatility': test_volas_for_Put_percent_3,
    '3_Day_Volatility': test_volas_for_Put_nominal_3,
    '9_Day_Percent_Volatility': test_volas_for_Put_percent_9,
    '9_Day_Volatility': test_volas_for_Put_nominal_9,
    '14_Day_Percent_Volatility': test_volas_for_Put_percent_14,
    '14_Day_Volatility': test_volas_for_Put_nominal_14,
    '21_Day_Percent_Volatility': test_volas_for_Put_percent_21,
    '21_Day_Volatility': test_volas_for_Put_nominal_21,
    '30_Day_Percent_Volatility': test_volas_for_Put_percent_30,
    '30_Day_Volatility': test_volas_for_Put_nominal_30,
    '60_Day_Percent_Volatility': test_volas_for_Put_percent_60,
    '60_Day_Volatility': test_volas_for_Put_nominal_60,
    '90_Day_Percent_Volatility': test_volas_for_Put_percent_90,
    '90_Day_Volatility': test_volas_for_Put_nominal_90,
    'Strike_1': testing_Features_for_Put_options['Strike_1'].values,
    'Close_1': testing_Data_Target_Put['Close_1'].values, 
    'Predicted Price_1': test_predictions_Put[:, 0],  
    'Strike_2': testing_Features_for_Put_options['Strike_2'].values,
    'Close_2': testing_Data_Target_Put['Close_2'].values,
    'Predicted Price_2': test_predictions_Put[:, 1],  
    'Strike_3': testing_Features_for_Put_options['Strike_3'].values,
    'Close_3': testing_Data_Target_Put['Close_3'].values,
    'Predicted Price_3': test_predictions_Put[:, 2], 
    'Strike_4': testing_Features_for_Put_options['Strike_4'].values,
    'Close_4': testing_Data_Target_Put['Close_4'].values,
    'Predicted Price_4': test_predictions_Put[:, 3], 
    'Strike_5': testing_Features_for_Put_options['Strike_5'].values,
    'Close_5': testing_Data_Target_Put['Close_5'].values,
    'Predicted Price_5': test_predictions_Put[:, 4], 
    'ETF Price on T - 5': testing_Features_for_Put_options['ETF Price on T - 5'].values,
})

for i in range(1, 6): 
    Test_Put_options[f'Strike_{i}'] = testing_Features_for_Put_options[f'Strike_{i}'].values
    Test_Put_options[f'Close_{i}'] = testing_Data_Target_Put[f'Close_{i}'].values  
    Test_Put_options[f'Predicted Price_{i}'] = test_predictions_Put[:, i - 1]  

for i in range(1, 6):
    Test_Put_options[f'Price Prediction Difference_{i}'] = Test_Put_options[f'Predicted Price_{i}'] - Test_Put_options[f'Close_{i}']
    
    Test_Put_options[f'Price Prediction Difference in percentage_{i}'] = (
        Test_Put_options[f'Price Prediction Difference_{i}'] / Test_Put_options[f'Close_{i}'].replace(0, np.nan) * 100
    )

Test_Put_options['Average Price Prediction Difference'] = Test_Put_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1)

Test_Put_options['Average Price Prediction Difference in percentage'] = Test_Put_options[
    [f'Price Prediction Difference in percentage_{i}' for i in range(1, 6)]
].mean(axis=1)

Test_Put_options['Overall Average Price Prediction Difference (APPDN)'] = Test_Put_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1).mean()

Test_Put_options['Overall Average Price Prediction Difference in percentage (APPDP)'] = Test_Put_options[
    [f'Price Prediction Difference in percentage_{i}' for i in range(1, 6)]
].mean(axis=1).mean()

metrics_df = pd.DataFrame({
    'Strike': [f'Strike_{i+1} Close_{i+1}' for i in range(len(MSE_test_Put))],
    'MSE_test_Put_close': MSE_test_Put,
    'MAE_test_Put_close': MAE_test_Put,
    'MAPE_test_Put_Close': MAPE_test_Put,

})

Test_Put_options['MSE'] = np.mean(MSE_test_Put) 
Test_Put_options['MAE'] = np.mean(MAE_test_Put) 
Test_Put_options['MAPE'] = np.mean(MAPE_test_Put)  
Res_Put_options['Learning rate'] = lr
Res_Put_options['Epoch size'] = Num_Epochs
Res_Put_options['Random seed'] = seed

print(Test_Put_options.head())
print(f'MSE: {np.mean(MSE_test_Put):.4f}, MAE: {np.mean(MAE_test_Put):.4f}, MAPE: {np.mean(MAPE_test_Put):.2f}%')



Test_Put_options = pd.concat([Test_Put_options, metrics_df], axis=1)

with pd.ExcelWriter('./testing_results/GLD_testing_results_with_MLP_sgd_multiple_strikes_Testing_10600_with_90d_volatility_input_Puts.xlsx') as writer:
    Test_Put_options.to_excel(writer, sheet_name='Put_Options', index=False)

print(Test_Put_options.head()) 


Put Options Price_1 - Strike_1: MSE (testing): 0.0586, MAE (testing): 0.1733, MAPE (testing): 38.37%
Put Options Price_2 - Strike_2: MSE (testing): 0.0740, MAE (testing): 0.1935, MAPE (testing): 26.10%
Put Options Price_3 - Strike_3: MSE (testing): 0.1025, MAE (testing): 0.2296, MAPE (testing): 21.28%
Put Options Price_4 - Strike_4: MSE (testing): 0.1306, MAE (testing): 0.2279, MAPE (testing): 14.51%
Put Options Price_5 - Strike_5: MSE (testing): 0.1665, MAE (testing): 0.2617, MAPE (testing): 12.85%
  Exp Date = T      T - 5  3_Day_Percent_Volatility  3_Day_Volatility  \
0   2016-01-15 2016-01-08              1.8628394824     18.0429640170   
1   2016-01-15 2016-01-08              1.8628394824     18.0429640170   
2   2016-01-22 2016-01-15              2.4371662270     23.2483473373   
3   2016-01-22 2016-01-15              2.4371662270     23.2483473373   
4   2016-01-29 2016-01-22              1.5853232806     15.2562142619   

   9_Day_Percent_Volatility  9_Day_Volatility  14_Day_Pe

/tmp/ipykernel_18705/3670883926.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_Put_loaded.load_state_dict(torch.load('./training_results/Puts.pth'))


In [37]:
strikes = [1, 2, 3, 4, 5]

for strike in strikes:
    plt.figure(figsize=(12, 6))
    
    plt.plot(range(len(Res_Call_options)), Res_Call_options[f'Close_{strike}'], 
             label=f'Actual Close Price Close_{strike} (Strike_{strike})', color='blue', marker='o', linestyle='-')
    
    plt.plot(range(len(Res_Call_options)), Res_Call_options[f'Predicted Price_{strike}'], 
             label=f'Predicted Close Price Close_{strike} (Strike_{strike})', color='red', marker='x', linestyle='--')
    
    plt.xlabel('Day')
    plt.ylabel('Close Price in USD$')
    plt.title(f'Actual vs Predicted Close Prices MLP 8 layers 9555 neurons for Call Options GLD ETF Training Close_{strike} with volatility input ', fontsize=10)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./training_results/Actual_vs_Predicted_MLP 8 layers 9555 neurons_Close_Prices_Close_{strike}Strike_Training{strike} with volatility input.png')
    plt.show()


for strike in strikes:
    plt.figure(figsize=(14, 6))
    plt.scatter(Res_Call_options.index, Res_Call_options[f'Price Prediction Difference_{strike}'], color='purple', alpha=0.5)
    plt.axhline(0, color='red', linestyle='--', linewidth=1)
    plt.xlabel('Day')
    plt.ylabel(f'Price Prediction Difference (Strike_{strike}) in USD$')
    plt.title(f'Residuals for Call Option Close Prices MLP 8 layers 9555 neurons GLD ETF Training dataset Close_{strike} with volatility input', fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./training_results/Residuals_Call_Options_MLP 8 layers 9555 neurons_Strike_Training{strike}_Close_{strike}with volatility input.png')
    plt.show()

plt.figure(figsize=(10, 6))
for strike in strikes:
    plt.scatter(Res_Call_options[f'Close_{strike}'], Res_Call_options[f'Predicted Price_{strike}'], 
                label=f'Strike_{strike} Close_{strike}', alpha=0.5)

min_price = Res_Call_options[[f'Close_{s}' for s in strikes]].min().min()
max_price = Res_Call_options[[f'Close_{s}' for s in strikes]].max().max()
plt.plot([min_price, max_price], [min_price, max_price], color='red', linewidth=2, linestyle='--', label='Perfect Prediction Line')

plt.xlabel('Actual Close Price in USD$')
plt.ylabel('Predicted Close Price in USD$')
plt.title('Actual vs Predicted Close Prices for Call Options in USD$ GLD ETF MLP 8 layers 9555 neurons Training dataset with volatility input', fontsize=10)
plt.legend()
plt.grid(True)

plt.savefig('./training_results/Actual_vs_Predicted_Close_Prices_MLP_8 layers 9555 neurons Scatter_Training_with volatility input.png')
plt.show()

plt.figure(figsize=(12, 6))

for strike in strikes:
    plt.figure(figsize=(10, 6))  
    plt.plot(dates_for_Call, Res_Call_options[f'Close_{strike}'], 
             label=f'Actual Close Price Strike_{strike}', marker='o', color='blue')
    plt.plot(dates_for_Call, Res_Call_options[f'Predicted Price_{strike}'], 
             label=f'Predicted Close Price Strike_{strike}', linestyle='--', marker='x', color='red')

    plt.xlabel('T - 5 Time years')
    plt.ylabel('Close Price in USD$')
    plt.title(f'Actual vs. Predicted Close prices GLD ETF MLP 8 layers 9555 neurons Over Time for Call Options Training dataset Close_{strike} with volatility input', fontsize=10)
    plt.xticks(rotation=45)  
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(f'./training_results/Actual_vs_Predicted_Close_{strike}_8 layers 9555 neurons MLP_Over_Time_Strike_Training{strike}_with volatility input.png')
    plt.show()
    
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(Res_Call_options['T - 5'], Res_Call_options['Overall Average Price Prediction Difference'], 
         marker='o', linestyle='-', color='blue')
plt.xlabel('T - 5 Time years')
plt.ylabel('Average Price Prediction Difference in USD$')
plt.title('Overall Average Price Prediction Difference 8 layers 9555 neurons MLP GLD for GLD ETF Call Options Training dataset with volatility input', fontsize=10)
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(Res_Call_options['T - 5'], Res_Call_options['Overall Average Price Prediction Difference in percentage'], 
         marker='o', linestyle='-', color='orange')
plt.xlabel('T - 5 Time years')
plt.ylabel('Average Price Prediction Difference in Percentage (%)')
plt.title('Overall Average Price Prediction Difference in Percentage 8 layers 9555 neurons MLP for GLD ETF Call Options Training dataset with volatility input', fontsize=10)
plt.grid(True)

plt.tight_layout()

plt.savefig('./training_results/Average_Price_Prediction_Differences_Training_with volatility input_MLP_8 layers 9555 neurons.png')

plt.show()

for strike in strikes:
    plt.plot(dates_for_Call, Res_Call_options[f'Close_{strike}'], 
             label=f'Actual Close Price Strike {strike}', marker='o')
    plt.plot(dates_for_Call, Res_Call_options[f'Predicted Price_{strike}'], 
             label=f'Predicted Close Price Close_{strike} Strike_{strike}', linestyle='--', marker='x')

plt.xlabel('T - 5 Time years')
plt.ylabel('Close Price in USD$')
plt.title('Actual vs. Predicted Close Prices Over Time for GLD ETF Call Options 8 layers 9555 neurons MLP Training dataset with volatility input', fontsize=10)
plt.xticks(rotation=45) 
plt.grid(True)
plt.tight_layout()

plt.savefig('./training_results/Actual_vs_Predicted_Close_Prices_Over_Time_8 layers 9555 neurons_MLP_Training_with_volatility.png')
plt.show()

Res_Call_options['Overall Price Prediction Difference'] = Res_Call_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1)

for strike in strikes:
    plt.figure(figsize=(12, 6))
    plt.hist(Res_Call_options[f'Price Prediction Difference_{strike}'], bins=30, color='green', alpha=0.7)
    plt.xlabel(f'Close_{strike} Price Prediction Difference in USD$ (Strike_{strike}) ')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Close Price Prediction Differences in USD$ Close_{strike} for GLD ETF Call Options 8 layers 9555 neurons MLP Training dataset with volatility input', fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./training_results/Prediction_Differences_Histogram_Call_Options_Close_{strike}_Strike_Training{strike}_with_volatility_input_8 layers 9555 neurons.png')
    plt.show()

plt.figure(figsize=(12, 6))
plt.hist(Res_Call_options['Overall Price Prediction Difference'], bins=30, color='blue', alpha=0.7)
plt.xlabel('Overall Price Prediction Difference in USD$')
plt.ylabel('Frequency')
plt.title('Histogram of Overall Prediction Differences in USD$ for GLD ETF Call Options 8 layers 9555 neurons MLP Training dataset with volatility input', fontsize=10)
plt.grid(True)
plt.tight_layout()

plt.savefig('./training_results/Prediction_Differences_Histogram_Call_Options_Overall_8 layers 9555 neurons_MLP_Training_with_volatility_input.png')
plt.show()

Res_Call_options['Overall Price Prediction Difference in percentage'] = Res_Call_options[
    [f'Price Prediction Difference in percentage_{i}' for i in range(1, 6)]
].mean(axis=1)

plt.figure(figsize=(12, 6))
plt.hist(Res_Call_options['Overall Price Prediction Difference in percentage'], bins=30, color='blue', alpha=0.7)
plt.xlabel('Overall Price Prediction Difference in %')
plt.ylabel('Frequency')
plt.title('Histogram of Overall Prediction Differences in percentage for GLD ETF Call Options 8 layers 9555 neurons MLP Training dataset with volatility input', fontsize=10)
plt.grid(True)
plt.tight_layout()

plt.savefig('./training_results/Prediction_Differences_in_percentage_Histogram_Call_Options_Overall_8 layers 9555 neurons_MLP_Training_with_volatility_input.png')
plt.show()


NameError: name 'Res_Call_options' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
strikes_test = [1, 2, 3, 4, 5]  

for strike in strikes_test:
    plt.figure(figsize=(12, 6))
    
    plt.plot(range(len(Test_Call_options)), Test_Call_options[f'Close_{strike}'], 
             label=f'Actual Close Price_{strike}(Strike_{strike})', color='blue', marker='o', linestyle='-')
    
    plt.plot(range(len(Test_Call_options)), Test_Call_options[f'Predicted Price_{strike}'], 
             label=f'Predicted Close Price_{strike} (Strike_{strike})', color='red', marker='x', linestyle='--')
    
    plt.xlabel('Day')
    plt.ylabel('Close Price in USD$')
    plt.title(f'Actual vs Predicted Close Prices for GLD ETF Call Options MLP 8 layers 9555 neurons Testing dataset Close_{strike} with volatility input', fontsize=10)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./testing_results/Actual_vs_Predicted_Close_Prices_Close_{strike}_Testing MLP with volatility input 8 layers 9555 neurons.png')
    plt.show()

for strike in strikes_test:
    plt.figure(figsize=(12, 6))
    plt.hist(Test_Call_options[f'Price Prediction Difference_{strike}'], bins=30, color='green', alpha=0.7)
    plt.xlabel(f'Price Prediction Difference in USD$ (Strike_{strike}) Close price_{strike}')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Prediction Differences in USD$ for GLD ETF Call Options 8 layers 9555 neurons MLP Testing dataset Close_{strike} with volatility input', fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./testing_results/Prediction_Differences_Histogram_Call_Options_MLP_Testing_Close_{strike} with volatility input.png')
    plt.show()

for strike in strikes_test:
    plt.figure(figsize=(14, 6))
    plt.scatter(Test_Call_options.index, Test_Call_options[f'Price Prediction Difference_{strike}'], color='purple', alpha=0.5)
    plt.axhline(0, color='red', linestyle='--', linewidth=1)
    plt.xlabel('Day')
    plt.ylabel(f'Price Prediction Difference in USD$ Close Price_{strike} in USD$')
    plt.title(f'Residuals for GLD ETF Call Option Close Prices Testing dataset Close_{strike} 8 layers 9555 neurons MLP with volatility input', fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'./testing_results/Residuals_Call_Options_Strike{strike}_Testing_Close_{strike}_MLP_with volatility input.png')
    plt.show()

plt.figure(figsize=(10, 6))
for strike in strikes_test:
    plt.scatter(Test_Call_options[f'Close_{strike}'], Test_Call_options[f'Predicted Price_{strike}'], 
                label=f'Strike_{strike} Close_{strike}', alpha=0.5)

min_price = Test_Call_options[[f'Close_{s}' for s in strikes_test]].min().min()
max_price = Test_Call_options[[f'Close_{s}' for s in strikes_test]].max().max()
plt.plot([min_price, max_price], [min_price, max_price], color='red', linewidth=2, linestyle='--', label='Perfect Prediction Line')

plt.xlabel('Actual Close Price in USD$')
plt.ylabel('Predicted Close Price in USD$')
plt.title('Actual vs Predicted Close Prices for GLD ETF Call Options in USD$ 8 layers 9555 neurons MLP Testing dataset with volatility input', fontsize=10)
plt.legend()
plt.grid(True)

plt.savefig('./testing_results/Actual_vs_Predicted_Close_Prices_Scatter_MLP_Testing_with volatility input.png')
plt.show()

plt.figure(figsize=(12, 6))

for strike in strikes_test:
    plt.figure(figsize=(10, 6)) 
    plt.plot(test_dates_for_Call, Test_Call_options[f'Close_{strike}'], 
             label=f'Actual Close Price Strike_{strike}', marker='o', color='blue')
    plt.plot(test_dates_for_Call, Test_Call_options[f'Predicted Price_{strike}'], 
             label=f'Predicted Close Price Strike_{strike}', linestyle='--', marker='x', color='red')

    plt.xlabel('T - 5 Time years')
    plt.ylabel('Close Price in USD$')
    plt.title(f'Actual vs. Predicted Close Prices Over Time for  GLD ETF Call Options 8 layers 9555 neurons MLP Testing dataset Close_{strike} with volatility input', fontsize=10)
    plt.xticks(rotation=45)  
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(f'./testing_results/Actual_vs_Predicted_Close_Prices_Over_Time_Strike_8 layers 9555 neurons MLP_Testing_{strike}_with volatility input.png')
    plt.show()
    
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(Test_Call_options['T - 5'], Test_Call_options['Overall Average Price Prediction Difference'], 
         marker='o', linestyle='-', color='blue')
plt.xlabel('T - 5 Time years')
plt.ylabel('Average Price Prediction Difference in USD$')
plt.title('Overall Average Price Prediction Difference for Call Options 8 layers 9555 neurons MLP Testing dataset with volatility input', fontsize=14)
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(Test_Call_options['T - 5'], Test_Call_options['Overall Average Price Prediction Difference in percentage'], 
         marker='o', linestyle='-', color='orange')
plt.xlabel('T - 5 Time in years')
plt.ylabel('Average Price Prediction Difference in Percentage (%)')
plt.title('Overall Average Price Prediction Difference in Percentage for GLD ETF Call Options 8 layers 9555 neurons MLP Testing dataset with volatility input', fontsize=14)
plt.grid(True)

plt.tight_layout()

plt.savefig('./testing_results/Average_Price_Prediction_Differences_ 8 layers 9555 neurons MLP_Testing_with volatility input.png')

plt.show()

Test_Call_options['Overall Price Prediction Difference'] = Test_Call_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1)



plt.figure(figsize=(12, 6))
plt.hist(Test_Call_options['Overall Price Prediction Difference'], bins=30, color='blue', alpha=0.7)
plt.xlabel('Overall Price Prediction Difference in USD$')
plt.ylabel('Frequency')
plt.title('Histogram of Overall Prediction Differences in USD$ for GLD ETF Call Options 8 layers 9555 neurons MLP Testing dataset with volatility input', fontsize=10)
plt.grid(True)
plt.tight_layout()

plt.savefig('./testing_results/Prediction_Differences_Histogram_Call_Options_Overall_8 layers 9555 neurons MLP_Testing_with_volatility_input.png')
plt.show()




Res_Call_options['Overall Price Prediction Difference in percentage'] = Res_Call_options[
    [f'Price Prediction Difference_{i}' for i in range(1, 6)]
].mean(axis=1)

plt.figure(figsize=(12, 6))
plt.hist(Res_Call_options['Overall Price Prediction Difference in percentage'], bins=30, color='blue', alpha=0.7)
plt.xlabel('Overall Price Prediction Difference %')
plt.ylabel('Frequency')
plt.title('Histogram of Overall Prediction Differences in percentage for GLD ETF Call Options 8 layers 9555 neurons MLP Testing dataset with volatility input', fontsize=10)
plt.grid(True)
plt.tight_layout()

plt.savefig('./training_results/Prediction_Differences_Histogram_Call_in_percentage_Options_Overall_MLP_Testing_with_volatility_input.png')
plt.show()